Proviamo a scrivere un modello del KP01

$\max \sum_{j=1,\dots,n} p_j x_j$

$\sum_{j=_1,\dots,n} w_j x_j \le W$

$x_j \in \{0,1\} ~~~ j=1,\dots,n$

e risolverlo per questa istanza:

oggetti = calzini,pantaloni,fiammiferi,spazzolino,mattone

pesi = [7,4,1,4,10]

profitti = [7,6,2,10,1]

W=14



Importiamo Pyomo e GLPK

In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 2.6 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 128216 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-2_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-2) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack ...

Importiamo dall'environment

In [ ]:

from pyomo.environ import * #qui andrebbe a seguire un nomo che diventi quella che usiamo nel codice, best practice


Adesso scriviamo il modello

Scriviamo un **concrete model**, ovvero siamo nel caso in cui i dati sono a disposizione nel momento in cui creo il modello

Innanzi tutto copiamo i dati dell'istanza in modo strutturato per poterli poi usare nel modello

In [ ]:
oggetti = ['calzini','pantaloni','fiammiferi','spazzolino','mattone']
pesi = {'calzini':7,'pantaloni':4,'fiammiferi':1,'spazzolino':4,'mattone':10} #utilizziamo le liste per assegnare in peso agli oggetti
profitti = {'calzini':7,'pantaloni':6,'fiammiferi':2,'spazzolino':10,'mattone':1} #dizionari
W = 14

Creo un modello *concrete* e lo chiamo KP01 (posso chiamarlo come voglio)

In [ ]:
KP01 = ConcreteModel() #KP01=Modello KnapSack
#add variables to the model KP01

Adesso che KP01 indica il mio modello, definisco quali sono le variabili. Ho una variabile per ogni elemento della lista *oggetti*; inoltre le variabili sono binarie

In [ ]:
KP01.variabili = Var(oggetti, domain=Binary) #le mie variabili sono gli oggetti(iterabili dalla lista). 
                                            #Il domain indforma il modello del fatto che le variabili sono binarie
                                            #variabili è un nome che diamo noi

Inserisco la funzione obiettivo usando la parola chiave *Objective* (in maiuscolo). Nella espressione (*expr*) itero su ogni elementi $i |in oggetti$, e in particolare per un certo oggetto, ad esempio il mattone, il dizionario *profitti[mattone]* mi restituisce il profitto dell'oggetto mattone.

In [ ]:
KP01.obiettivo = Objective(expr = sum(profitti[i]*KP01.variabili[i] for i in oggetti), sense = maximize) #i è la chiave che uso per accedere al valore del dizionario
                                  #

Vincoli

In modo simile aggiungo un vincolo con la parola chiave *Constraint*. Il fatto che si chiami KP01.cap piuttosto che KP01.vincolo è una mia scelta.

In [ ]:
KP01.cap = Constraint(expr = sum(pesi[i]*KP01.variabili[i] for i in oggetti) <=W) #lo chiamo KP01.cap perche è un vincolo di capacita

Quindi questo modello è sempicemente un modello a cui stiamo assegnando delle espressioni, dopo le utilizzeremo in un costrutto

In [ ]:
opt = SolverFactory('glpk')  #con dei numeri molto grandi e complessi, questo va un po in crisi, bisognerebbe usarne uno piu performante

In [ ]:
opt.solve(KP01, tee=True) #tee è un parametro che dice che voglio che mi stampi a video dei valori cosicchè li possa vedere

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpi6qg4tur.glpk.raw --wglp /tmp/tmpz0meeh1i.glpk.glp --cpxlp
 /tmp/tmpe_ppur46.pyomo.lp
Reading problem data from '/tmp/tmpe_ppur46.pyomo.lp'...
/tmp/tmpe_ppur46.pyomo.lp:31: warning: lower bound of variable 'x1' redefined
/tmp/tmpe_ppur46.pyomo.lp:31: warning: upper bound of variable 'x1' redefined
2 rows, 6 columns, 6 non-zeros
5 integer variables, all of which are binary
36 lines were read
Writing problem data to '/tmp/tmpz0meeh1i.glpk.glp'...
25 lines were written
GLPK Integer Optimizer, v4.65
2 rows, 6 columns, 6 non-zeros
5 integer variables, all of which are binary
Preprocessing...
1 row, 5 columns, 5 non-zeros
5 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+01  ratio =  1.000e+01
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
Solving LP relaxation...
GLPK Simplex Optimizer, v4.65
1 row,

{'Problem': [{'Name': 'unknown', 'Lower bound': 19.0, 'Upper bound': 19.0, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 6, 'Number of nonzeros': 6, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '5', 'Number of created subproblems': '5'}}, 'Error rc': 0, 'Time': 0.024311065673828125}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:
KP01.pprint()

1 Set Declarations
    variabili_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'calzini', 'pantaloni', 'fiammiferi', 'spazzolino', 'mattone'}

1 Var Declarations
    variabili : Size=5, Index=variabili_index
        Key        : Lower : Value : Upper : Fixed : Stale : Domain
           calzini :     0 :   1.0 :     1 : False : False : Binary
        fiammiferi :     0 :   1.0 :     1 : False : False : Binary
           mattone :     0 :   0.0 :     1 : False : False : Binary
         pantaloni :     0 :   0.0 :     1 : False : False : Binary
        spazzolino :     0 :   1.0 :     1 : False : False : Binary

1 Objective Declarations
    obiettivo : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 7*variabili[calzini] + 6*variabili[pantaloni] + 2*variabili[fiammiferi] + 10*variabili[spazzolino] + variabili[mattone]

1 Constraint Declarat

In [ ]:
KP01.display() #fa vedere solo quello che mi interessa

Model unknown

  Variables:
    variabili : Size=5, Index=variabili_index
        Key        : Lower : Value : Upper : Fixed : Stale : Domain
           calzini :     0 :   1.0 :     1 : False : False : Binary
        fiammiferi :     0 :   1.0 :     1 : False : False : Binary
           mattone :     0 :   0.0 :     1 : False : False : Binary
         pantaloni :     0 :   0.0 :     1 : False : False : Binary
        spazzolino :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    obiettivo : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  19.0

  Constraints:
    cap : Size=1
        Key  : Lower : Body : Upper
        None :  None : 12.0 :  14.0
